In [3]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import display, Markdown
from openai import OpenAI

# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"


class Website():

    def __init__(self, url):
        self.url = url
        response = requests.get(url,headers=HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


def get_messages(website):
    system_prompt = "You are an assistant who analyzes the content of a website \
    and provides a short summary, ignoring the text that might be related to \
    navigation. Respond in markdown."

    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\n Please provide a short summary of this website in markdown. \
    If the website contains some list containing top news etc, please exclude them from your summary. \
    Give me the summary as bullet points, with each point not exceeding 2 sentences. \
    The content of the website are as follows: \n\n"
    user_prompt += website.text

    return [
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_prompt}
    ]

def get_payload(website):
    return {
        "model": MODEL,
        "messages": get_messages(website),
        "stream": False
    }

    
def summarize(url):
    website = Website(url)
    response = requests.post(OLLAMA_API, json=get_payload(website), headers=HEADERS)
    return response.json()['message']['content']
    

def display_summary(url):
    display(Markdown(summarize(url)))

display_summary("https://www.edwarddonner.com")


### Summary of the Website Home - Edward Donner

* **Founder and CTO of Nebula.io**: Ed is the co-founder and CTO of Nebula.io, a company applying AI to help people discover their potential. He previously founded and led an AI startup that was acquired in 2021.
* **Expertise and Interests**: Ed enjoys writing code, experimenting with LLMs, DJing, and amateur electronic music production. He is also interested in learning about new technologies on Hacker News.
* **Company Overview**: Nebula.io uses proprietary LLMs to help recruiters source, understand, engage, and manage talent. The company has patented its matching model and received press coverage for its award-winning platform.
* **Resources and Workshops**: Ed shares resources and workshop materials related to AI and LLM engineering, including topics such as mastering AI, getting started with agents, and transitioning from software engineering to data science.